In [122]:
import torch
import torch.nn as nn
from collections import OrderedDict

class Wide_Deep(nn.Module):
    def __init__(self, wide_dim, deep_dim, action_dim, embeddings={}, deep_neurons=[32, 16], activation=nn.ReLU()):
        
        super(Wide_Deep, self).__init__()
        self.wide_dim = wide_dim
        self.deep_dim = deep_dim
        self.context_dim = wide_dim + deep_dim
        self.action_dim = action_dim
        self.deep_neurons = deep_neurons

        self.add_module('wide', nn.Module())
        self.add_module('deep', nn.Module())

        dims = {'wide':wide_dim, 'deep':deep_dim}
        for name, child_module in self.named_children():
            if name in embeddings.keys():
                assert dims[name] >= len(embeddings[name]), "Number of {} embedding features defined in embeddings should not more than {}_dim.".format(name, name)
                for embed in embeddings[name]:
                    child_module.add_module(embed[0], nn.Embedding(embed[1], embed[2]))
                    dims[name] += -1
                    dims[name] += embed[2]  
        self.dims = dims

        if self.dims['deep'] == 0:
            assert (self.dims['wide'] > 0), "Both wide_dim and deep_dim are 0, at least one of them needs a positive value." 
            print("This is a wide-only model.")
            self.z_dim = self.dims['wide']
        else:
            if self.dims['wide'] == 0:
                print("This is a deep-only model.")
            else:
                print("This is a wide and deep model.")
            assert len(deep_neurons) > 0, "deep_neurons must not be empty for the deep part."
            deep_layers = OrderedDict()
            layer_in = self.dims['deep']
            for i, layer_out in enumerate(deep_neurons):
                deep_layers['fc{}'.format(i)] = nn.Linear(layer_in, layer_out)
                deep_layers['activ{}'.format(i)] = activation
                layer_in = layer_out
            self.deep.add_module('NN', nn.Sequential(deep_layers))
            self.z_dim = self.dims['wide'] + deep_neurons[-1]

        self.lastlayer = nn.Linear(self.z_dim, self.action_dim)

    def get_z(self, x):
        inputs = {'wide':x[:, :self.wide_dim], 'deep':x[:, self.wide_dim:]}
        
        after_embed = {}
        for name, module in self.named_children():
            if name not in ['wide', 'deep']:
                break;
            i = 0
            for child in module.children():
                if 'Embedding' in torch.typename(child):
                    if name not in after_embed:
                        after_embed[name] = child(inputs[name][:, i].long())
                        i += 1
                    else:
                        after_embed[name] = torch.cat((after_embed[name], child(inputs[name][:, i].long())), dim=1)
                        i += 1 
            if name not in after_embed:
                after_embed[name] = inputs[name]
            else:
                after_embed[name] = torch.cat((after_embed[name], inputs[name][:, i:]), dim=1)

        if self.dims['deep'] == 0:
            z = after_embed['wide']
        elif self.dims['wide'] == 0:
            z = self.deep.NN(after_embed['deep'])
        else:
            z = torch.cat((after_embed['wide'], self.deep.NN(after_embed['deep'])), dim=1)
        
        return z

    def forward(self, x):
        z = self.get_z(x)
        out = self.lastlayer(z)
        return out
                
            

In [123]:
embeddings={'wide':[['feature_1', 100, 64],
                    ['feature_2', 200, 64]]
            }

wide_dim = 10
deep_dim = 0
action_dim = 3
testmodel = Wide_Deep(wide_dim, deep_dim, action_dim, embeddings=embeddings)
testmodel

This is a wide-only model.


Wide_Deep(
  (wide): Module(
    (feature_1): Embedding(100, 64)
    (feature_2): Embedding(200, 64)
  )
  (deep): Module()
  (lastlayer): Linear(in_features=136, out_features=3, bias=True)
)

In [124]:
embeddings={'deep':[['feature_3', 300, 64],
                    ['feature_4', 400, 64],
                    ['feature_5', 500, 64]]
            }

wide_dim = 0
deep_dim = 12
action_dim = 3
testmodel = Wide_Deep(wide_dim, deep_dim, action_dim, embeddings=embeddings)
testmodel

This is a deep-only model.


Wide_Deep(
  (wide): Module()
  (deep): Module(
    (feature_3): Embedding(300, 64)
    (feature_4): Embedding(400, 64)
    (feature_5): Embedding(500, 64)
    (NN): Sequential(
      (fc0): Linear(in_features=201, out_features=32, bias=True)
      (activ0): ReLU()
      (fc1): Linear(in_features=32, out_features=16, bias=True)
      (activ1): ReLU()
    )
  )
  (lastlayer): Linear(in_features=16, out_features=3, bias=True)
)

In [125]:
embeddings={'wide':[['feature_1', 100, 64],
                    ['feature_2', 200, 64]],
            'deep':[['feature_3', 300, 64],
                    ['feature_4', 400, 64],
                    ['feature_5', 500, 64]]
            }

wide_dim = 10
deep_dim = 12
action_dim = 3
testmodel = Wide_Deep(wide_dim, deep_dim, action_dim, embeddings=embeddings)
testmodel

This is a wide and deep model.


Wide_Deep(
  (wide): Module(
    (feature_1): Embedding(100, 64)
    (feature_2): Embedding(200, 64)
  )
  (deep): Module(
    (feature_3): Embedding(300, 64)
    (feature_4): Embedding(400, 64)
    (feature_5): Embedding(500, 64)
    (NN): Sequential(
      (fc0): Linear(in_features=201, out_features=32, bias=True)
      (activ0): ReLU()
      (fc1): Linear(in_features=32, out_features=16, bias=True)
      (activ1): ReLU()
    )
  )
  (lastlayer): Linear(in_features=152, out_features=3, bias=True)
)

In [126]:
embeddings = {}

wide_dim = 10
deep_dim = 12
action_dim = 3
testmodel = Wide_Deep(wide_dim, deep_dim, action_dim, embeddings=embeddings)
testmodel

This is a wide and deep model.


Wide_Deep(
  (wide): Module()
  (deep): Module(
    (NN): Sequential(
      (fc0): Linear(in_features=12, out_features=32, bias=True)
      (activ0): ReLU()
      (fc1): Linear(in_features=32, out_features=16, bias=True)
      (activ1): ReLU()
    )
  )
  (lastlayer): Linear(in_features=26, out_features=3, bias=True)
)

**Note:**
1. `wide_dim` and `deep_dim` should NOT be `0` at the same time, other wise the model will raise an error.
2. Number of `embeddings` of each part (wide/deep) should not larger than the input dimension (`wide_dim`/`deep_dim`) of that part, otherwise the model will raise an error.
3. if `deep_dim` > 0, `deep_neurons` should not be empty.


In [127]:
embeddings={'wide':[['feature_1', 100, 64],
                    ['feature_2', 200, 64]],
            'deep':[['feature_3', 300, 64],
                    ['feature_4', 400, 64],
                    ['feature_5', 500, 64]]
            }

wide_dim = 10
deep_dim = 12
action_dim = 3
testmodel = Wide_Deep(wide_dim, deep_dim, action_dim, embeddings=embeddings)
testmodel

This is a wide and deep model.


Wide_Deep(
  (wide): Module(
    (feature_1): Embedding(100, 64)
    (feature_2): Embedding(200, 64)
  )
  (deep): Module(
    (feature_3): Embedding(300, 64)
    (feature_4): Embedding(400, 64)
    (feature_5): Embedding(500, 64)
    (NN): Sequential(
      (fc0): Linear(in_features=201, out_features=32, bias=True)
      (activ0): ReLU()
      (fc1): Linear(in_features=32, out_features=16, bias=True)
      (activ1): ReLU()
    )
  )
  (lastlayer): Linear(in_features=152, out_features=3, bias=True)
)

In [128]:
# A simple test on the get_z and forward functions.

batch_size = 16
embed_inputs = {}
embed_inputs['feature_1'] = torch.tensor(np.random.randint(100, size=(batch_size, 1))).float()
embed_inputs['feature_2'] = torch.tensor(np.random.randint(200, size=(batch_size, 1))).float()
embed_inputs['feature_3'] = torch.tensor(np.random.randint(300, size=(batch_size, 1))).float()
embed_inputs['feature_4'] = torch.tensor(np.random.randint(400, size=(batch_size, 1))).float()
embed_inputs['feature_5'] = torch.tensor(np.random.randint(500, size=(batch_size, 1))).float()

wide_others = torch.tensor(np.random.random(size=(batch_size, wide_dim-2))).float()
deep_others = torch.tensor(np.random.random(size=(batch_size, deep_dim-3))).float()
wide_inputs = torch.cat((embed_inputs['feature_1'], embed_inputs['feature_2'], wide_others), dim=1)
deep_inputs = torch.cat((embed_inputs['feature_3'], embed_inputs['feature_4'], embed_inputs['feature_5'], deep_others), dim=1)

inputs = torch.cat((wide_inputs, deep_inputs), dim=1)
zs = testmodel.get_z(inputs)
print(zs.shape)
outs = testmodel(inputs)
print(outs.shape)

torch.Size([16, 152])
torch.Size([16, 3])
